In [9]:
import torch
import torch.nn as nn
from dataclasses import dataclass

import requests
import unicodedata

# Classes

In [10]:
@dataclass
class Config:
    d_model: int
    d_vocab: int
    d_hidden: int
    n_context: int

In [11]:
# class Embedding(nn.Module):
#     def __init__(self):
#         super().__init__()
    
#     def forward(self):
#         pass

class Attention(nn.Module):
    def __init__(self, config: Config):
        super().__init__()
        # self.W_qk = nn.Linear(config.d_model, config.d_vocab)
        self.bilinear = nn.Bilinear(config.d_model, config.d_model, config.n_context, bias=False)
        self.M = torch.triu(torch.ones((config.n_context, config.n_context)), diagonal=1)
        self.M = self.M.masked_fill(self.M.bool(), -torch.inf)
        self.second_matmult = nn.Linear(config.d_model, config.d_model, bias=False)
        self.softmax = nn.Softmax()
    
    def forward(self, x):
        xwx = self.bilinear(x, x) # d_m x d_m
        x_masked = xwx+ self.M 
        x_softmaxed = self.softmax(x_masked)
        x_fin = x_softmaxed@x
        #multiply softmaxed by x
        #multiply that by wov
        x_fin = self.second_matmult(x_fin)
        return x_fin

class MLP(nn.Module):
    def __init__(self, config: Config):
        super().__init__()
        self.linear_up = nn.Linear(config.d_model, config.d_hidden)
        self.linear_down = nn.Linear(config.d_hidden, config.d_model)
    
    def forward(self, x):
        x = self.linear_up(x)
        x = torch.relu(x)
        x = self.linear_down(x)
        return x
    
class TransformerBlock(nn.Module):
    def __init__(self, config: Config):
        super().__init__()
        self.config = config

        self.MLP = MLP(config=self.config)
        self.Attention = Attention(config=self.config)
    
    def forward(self, x):
        return x + self.Attention(x) + self.MLP(x)

$n_c$: Context window length

$d_m$: Model Dimension

$d_v$: Vocab Dimension

In [12]:
text_sample = "The quick brown fox jumped over the lazy dog."

In [13]:
d_model = 10
d_vocab = 10
d_hidden = 10
n_context = 10

conf = Config(d_model, d_vocab, d_hidden, n_context)

embedding = nn.Embedding(num_embeddings=conf.d_vocab, embedding_dim=conf.d_model)



# Tokenization Code

In [20]:
from pathlib import Path

def get_gutenberg_book(
	id: int | None = 84,
	data_temp: Path | str = "../data/gutenberg_data",
	remove_gutenberg_meta: bool = True,
) -> str:
	
	data_temp: Path = Path(data_temp)
	data_temp.mkdir(parents=True, exist_ok=True)
	
	url: str = f"https://www.gutenberg.org/cache/epub/{id}/pg{id}.txt"
	data_path: Path = Path(data_temp) / f"{id}.txt"
	data: str
	# read from cache if it exists
	if data_path.exists():
		with open(data_path, 'r', encoding='utf-8') as file:
			data = file.read()
	else:
		# download if it doesn't exist
		response: requests.Response = requests.get(url)
		response.raise_for_status()  # Ensure that the download was successful
		data = response.text

		# save to cache
		with open(data_path, 'w', encoding='utf-8') as file:
			file.write(data)

	# remove header/footer
	if remove_gutenberg_meta:
		data = '***'.join(data.split('***')[2:])
		data = '***'.join(data.split('***')[:-1])
	
	return data

def get_many_books(
		ids: list[int],
		data_temp: Path | str = "../data/gutenberg_data",
	) -> list[str]:
	
	data: list[str] = []
	for id in ids:
		print(f"Getting book {id}...")
		item: str = get_gutenberg_book(id, data_temp)
		print(f"\t{len(item)} characters read")
		data.append(item)
	
	return data

In [15]:
def process_text(
	text: str,
	allowed_punctuation: str = "-.,;:!?()\"\\" + "".join(str(x) for x in range(10)),
	punctuation_convert: dict[str, str] = {'—': '-'},
) -> str:
	
	# replace some special characters which unicode won't normalize properly
	for char, replacement in punctuation_convert.items():
		text = text.replace(char, replacement)

	# if a line has ".jpg" in it, remove that line (this is specific to Don Quixote)
	text = '\n'.join(
		line 
		for line in text.split('\n')
		if '.jpg' not in line
	)

	# Normalize the string to decompose Unicode characters
	text = unicodedata.normalize('NFKD', text)

	# Encode to ASCII bytes, then decode back to string, ignoring errors
	text = text.encode('ascii', 'ignore').decode('ascii')

	# remove newlines and tabs
	text = text.replace('\n', ' ').replace('\t', ' ')


	# put spaces around allowed punctuation
	for char in allowed_punctuation:
		text = text.replace(char, f' {char} ')


	# remove leading and trailing spaces
	text = text.strip()

	# remove multiple spaces
	while '  ' in text:
		text = text.replace('  ', ' ')


	# remove all characters except (alphanumeric, allowed_punctuation, ' ')
	text = ''.join(
		(
			char 
			if (
				char.isalnum() 
				or char in allowed_punctuation 
				or char == ' '
			)
			else ' '
		)
		for char in text 
	)

	# convert to lowercase
	text = text.lower()

	text = text.strip()

	return text

In [16]:
def tokenize(
	text: str,
	process: bool = False,
) -> list[str]:
	if process:
		text = process_text(text)
	return text.split(' ')

In [17]:
DATA_RAW: list[str] = get_many_books([84, 15, 18, 82, 996, 2600])
DATA: str = " ".join(process_text(x) for x in DATA_RAW)
DATA_TOKENIZED: list[str] = tokenize(DATA)

Getting book 84...
	419422 characters read
Getting book 15...
	1238469 characters read
Getting book 18...
	1172825 characters read
Getting book 82...
	1103796 characters read
Getting book 996...
	2299352 characters read
Getting book 2600...
	3208337 characters read


# Tests

In [18]:
d_model = 10
d_vocab = 10
d_hidden = 10
n_context = 5

x = torch.randn((n_context, d_model))

conf = Config(d_model, d_vocab, d_hidden, n_context)
mlp = MLP(conf)
attention = Attention(conf)
Aoutput = attention(x)
print(Aoutput.shape)

output = mlp(x)
print(output)



torch.Size([5, 10])
tensor([[-0.3691,  0.0318,  0.1421, -0.2532, -0.5018, -0.1011,  0.2451, -0.0393,
          0.0351, -0.0041],
        [-0.2701,  0.1775,  0.0922, -0.2210, -0.3877, -0.0682,  0.2059, -0.2404,
          0.1205, -0.1657],
        [ 0.0494,  0.2298, -0.1569, -0.2315, -0.3186, -0.1338, -0.3310, -0.0626,
          0.0645,  0.0095],
        [ 0.0099,  0.2639, -0.0911, -0.2583, -0.3254, -0.1297, -0.2129, -0.2359,
          0.1237, -0.0108],
        [ 0.1208,  0.2542, -0.1770, -0.1454, -0.2068, -0.0533, -0.0912, -0.0022,
          0.0207,  0.2528]], grad_fn=<AddmmBackward0>)


/Users/danielkrawciw/Documents/llms/transformer/.venv/lib/python3.12/site-packages/torch/nn/modules/module.py:1776: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


In [19]:
# Transformer Block test

d_model = 10
d_vocab = 10
d_hidden = 10
n_context = 5

config = Config(
    d_model = d_model,
    d_vocab = d_vocab,
    d_hidden = d_hidden,
    n_context = n_context,
)

x = torch.randn((n_context, d_model))
conf = Config(d_model, d_vocab, d_hidden, n_context)

tb = TransformerBlock(config)

output_x = tb(x)
output_x


tensor([[-1.3712,  0.3981, -0.6220,  2.4504, -1.2197, -0.9029, -2.5249,  0.3925,
          0.1311,  0.9976],
        [-1.2218,  0.6218, -0.4370,  0.0209, -1.0655, -0.3094,  0.3904,  0.6515,
         -0.9897, -1.3531],
        [ 0.0074, -0.1127,  0.2799, -0.3389, -0.0927, -1.2503,  0.3873,  1.3605,
         -0.1407,  1.2934],
        [-0.8746,  2.0001,  0.8767,  2.9100, -1.4419, -0.8370,  0.6010,  0.5973,
          0.3860, -0.8884],
        [-1.0056,  0.7414, -1.6013, -0.4434, -1.5685, -0.4595, -0.5369,  0.1884,
         -0.8158, -0.8711]], grad_fn=<AddBackward0>)